In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
from rasterio import features
import numpy as np

In [2]:
def read_raster(path,bands=1,crs=False):
    # From rasterio docs with modifications
    with rio.open(path) as dst:
        array = dst.read(bands)
        profile = dst.profile
        crs_val = dst.crs
        # array = np.moveaxis(array,0,-1)
    
    result = [array, profile]

    if crs:
        result.append(crs_val)

    return result


def write_raster(array,profile,out_path,nodata,dtype):
    # From rasterio docs:
    # Register GDAL format drivers and configuration options with a
    # context manager.
    with rio.Env():
        # And then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile.update(
            dtype=dtype,
            count=1,
            nodata=nodata,
            compress='lzw')

        with rio.open(out_path, 'w', **profile) as dst:
            dst.write(array.astype(dtype), 1)

    return out_path

def polygon_to_raster(gdf,template_path,value=1,crs=False):
    if isinstance(gdf,str):
        pol = gpd.read_file(gdf)
    else:
        pol = gdf

    with rio.open(template_path) as dst:
        profile = dst.profile
        template_crs = dst.crs
        template_transform = profile['transform']
        template_shape = dst.shape

    # if crs != pol.crs:
    #   raise Exception('CRSs do not match!')

    geojsons = [x['geometry'] for x in pol.geometry.__geo_interface__['features']]
    if isinstance(value,str):
        shapes = [tuple(x) for x in zip(geojsons,pol[value])]
    else:
        shapes = [(x,value) for x in geojsons]

    array = features.rasterize(shapes, out_shape=template_shape, transform=template_transform)
    
    result = [array, profile]
    if crs:
        result.append(template_crs)

    return result

In [3]:
cd /home/micromamba/data

/home/micromamba/data


In [4]:
horizons = 'DoD/horizons_dsm_01022016.tif'
sfm =  'DoD/sfm_dsm_01092021_1m.tif'
error_gdf = gpd.read_file('DoD/polygons/error_zones.shp')
stable_gdf = gpd.read_file('DoD/polygons/stable_zones.shp')
landslide_gdf = gpd.read_file('DoD/polygons/landslide_zones.shp')

In [5]:
dem_paths = {'horizons':horizons,'sfm':sfm}
dems = {}

for r in dem_paths:
    ras_path = dem_paths[r]
    ras, profile = read_raster(ras_path)
    
    errors, _ = polygon_to_raster(error_gdf,ras_path,value=1,crs=False)
    stable, _ = polygon_to_raster(stable_gdf,ras_path,value=1,crs=False)
    
    ras[ras <= 0] = np.nan
    ras[errors == 1] = np.nan
    mean_val = np.nansum(ras[stable == 1]) / np.nansum(stable)
    
    dems[r] = {'array':ras,'profile':profile,'mean_val':mean_val}

In [6]:
offset = dems['sfm']['mean_val'] - dems['horizons']['mean_val']
offset

1.0096655611242795

In [7]:
dems['sfm']['array'] = dems['sfm']['array'] - offset

In [8]:
for d,p in zip(dems,['DoD/horizons_dsm_01022016_filter.tif', 'DoD/sfm_dsm_01092021_filter.tif']):

    dems[d]['array'][np.isnan(dems[d]['array'])] = 0

    write_raster(dems[d]['array'],dems[d]['profile'],p,nodata=0,dtype=rio.float32)

In [9]:
arr, profile = polygon_to_raster(landslide_gdf,horizons,value='id',crs=False)
arr = arr.astype(float)
# arr[arr == 0] = np.nan
write_raster(arr,profile,'DoD/landslide_zones.tif',nodata=0,dtype=rio.float32)

'DoD/landslide_zones.tif'